<a href="https://colab.research.google.com/github/andysingal/python-advanced/blob/main/basics/Pydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The pydantic Library
In this tutorial we're going to take a look at the pydantic library.

This is a great library for creating data models in Python, leveraging Python's built-in type hints.

I first came across this library when I started using FastAPI for API development.

Since then I have been using pydantic for way more than just FastAPI - I find myself using that library practically in almost every project I write.

It makes defining data structures (or models in Pydantic's terminology) with "strict typing" very easy. Furthermore, Pydantic provides data validation (what basically gives us "strict typing", in the sense that we are guaranteed that fields in the structure will be of some specific type), as well as the ability to easily serialize and deserialize data (Python dict objects, JSON and even supports pickling).

It also helps with IDE features such as inspection, type checking and auto-completion.

I find myself using Pydantic outside of FastAPI, both for the IDE benefits, as well as easy marshalling of data with NoSQL databases.

You will need to install pydantic into your virtual environment - see here - although if you use pipenv, the Pipfile included in this github repo already contains Pydantic.




In [6]:
from pydantic import BaseModel, ValidationError
from typing import Optional

In [2]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [3]:
p = Person(first_name='Isaac', last_name='Newton', age=84)

In [5]:
try:
    Person(first_name='Isaac')
except ValidationError as ex:
    print(ex)

2 validation errors for Person
last_name
  field required (type=value_error.missing)
age
  field required (type=value_error.missing)


In [7]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: Optional[int]

In [8]:
class Person(BaseModel):
    first_name: str = None
    last_name: str
    age: int = None

In [9]:
p = Person(first_name='Isaac', last_name='Newton', age=84)

In [10]:
p.dict(exclude={'first_name', 'age'})

{'last_name': 'Newton'}

In [11]:
print(p.json(include={'last_name', 'age'}, indent=4))

{
    "last_name": "Newton",
    "age": 84
}


In [12]:
from datetime import date

class Person(BaseModel):
    first_name: str = None
    last_name: str
    dob: date

In [13]:
data = {
    "first_name": "Isaac",
    "last_name": "Newton",
    "dob": date(1643, 1, 4)
}

In [14]:
p = Person.parse_obj(data)
p

Person(first_name='Isaac', last_name='Newton', dob=datetime.date(1643, 1, 4))